<a href="https://colab.research.google.com/github/sezinbiner/ClassificationOfCovidXray/blob/main/withKfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
 from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)
generator_dir = "/content/gdrive/MyDrive/Kaggle/COVID-19_Radiography_Dataset"

class_names = ['COVID', 'Normal']
generator = datagen.flow_from_directory(
        generator_dir,
        target_size=(224, 224),  
        batch_size=20,
        shuffle=True,
        class_mode='binary')


Found 13808 images belonging to 2 classes.


In [2]:
def extract_features(number_of_images, generator, batch_size):
  features = np.zeros(shape=(number_of_images, extraction_layer.output_shape[1], extraction_layer.output_shape[2], extraction_layer.output_shape[3]))
  labels = np.zeros(shape=(number_of_images))
  i = 0
  print("loop")
  for inputs_batch, labels_batch in generator:
      features_batch = extraction_layer.predict(inputs_batch)
      features[i * batch_size : (i + 1) * batch_size] = features_batch
      labels[i * batch_size : (i + 1) * batch_size] = labels_batch
      i += 1
      if i * batch_size >= number_of_images:
          break
  print("loop over")
  return features, labels

In [3]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import  InceptionV3
import numpy as np

extraction_layer = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)
extraction_layer.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
features,labels = extract_features(13808, generator, generator.batch_size)

loop
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

In [7]:
from keras.models import Sequential
from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(
    loss='binary_crossentropy',
    optimizer= "adam" ,
    metrics=['accuracy']
)

In [9]:
from sklearn.model_selection import KFold                    
kf = KFold(n_splits=10)
X = features
y = labels

In [11]:
index_list = []
test_index_list = []
i = 1
#Trying different train and test sets for higher accuracy
for train_index,test_index in kf.split(X):
    index_list.append(train_index)
    index_list.append(test_index)

In [12]:
index_list

[array([ 1381,  1382,  1383, ..., 13805, 13806, 13807]),
 array([   0,    1,    2, ..., 1378, 1379, 1380]),
 array([    0,     1,     2, ..., 13805, 13806, 13807]),
 array([1381, 1382, 1383, ..., 2759, 2760, 2761]),
 array([    0,     1,     2, ..., 13805, 13806, 13807]),
 array([2762, 2763, 2764, ..., 4140, 4141, 4142]),
 array([    0,     1,     2, ..., 13805, 13806, 13807]),
 array([4143, 4144, 4145, ..., 5521, 5522, 5523]),
 array([    0,     1,     2, ..., 13805, 13806, 13807]),
 array([5524, 5525, 5526, ..., 6902, 6903, 6904]),
 array([    0,     1,     2, ..., 13805, 13806, 13807]),
 array([6905, 6906, 6907, ..., 8283, 8284, 8285]),
 array([    0,     1,     2, ..., 13805, 13806, 13807]),
 array([8286, 8287, 8288, ..., 9664, 9665, 9666]),
 array([    0,     1,     2, ..., 13805, 13806, 13807]),
 array([ 9667,  9668,  9669, ..., 11045, 11046, 11047]),
 array([    0,     1,     2, ..., 13805, 13806, 13807]),
 array([11048, 11049, 11050, ..., 12425, 12426, 12427]),
 array([    0,  

In [ ]:
print("\n")
print(f"Fold : {i}")
print("Train:",index_list[0], "Test:", index_list[1])
i = i +1
X_train = X[index_list[4]]                           
X_test = X[index_list[5]]
y_train = y[index_list[4]]
y_test = y[index_list[5]]
print(len(X_train))



Fold : 4
Train: [ 1381  1382  1383 ... 13805 13806 13807] Test: [   0    1    2 ... 1378 1379 1380]


In [17]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=20,
                    validation_split=0.2)
from sklearn import metrics
y_pred = model.predict(X_test)
probas = np.array(y_pred)
labels = []
for prob in probas:
  if prob < 0.5:
    labels.append(0)
  else:
    labels.append(1)
y_true_labels = y_test
y_pred_labels = labels
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_true_labels, y_pred_labels)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_true_labels, y_pred_labels,pos_label='positive', average='micro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_true_labels, y_pred_labels, average='micro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')
print('F1 score: %f' % f1)
from sklearn.metrics import classification_report
print(classification_report(y_true_labels, y_pred_labels, target_names=["covid", "normal"]))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true_labels, y_pred_labels))


Epoch 1/10
498/498 [==============================] - 2s 5ms/step - loss: 0.1034 - accuracy: 0.9558 - val_loss: 0.0614 - val_accuracy: 0.9783
Epoch 2/10
498/498 [==============================] - 3s 5ms/step - loss: 0.0998 - accuracy: 0.9557 - val_loss: 0.0609 - val_accuracy: 0.9779
Epoch 3/10
498/498 [==============================] - 3s 5ms/step - loss: 0.0850 - accuracy: 0.9638 - val_loss: 0.0599 - val_accuracy: 0.9783
Epoch 4/10
498/498 [==============================] - 2s 4ms/step - loss: 0.0795 - accuracy: 0.9667 - val_loss: 0.0873 - val_accuracy: 0.9751
Epoch 5/10
498/498 [==============================] - 2s 5ms/step - loss: 0.0856 - accuracy: 0.9635 - val_loss: 0.0557 - val_accuracy: 0.9783
Epoch 6/10
498/498 [==============================] - 2s 4ms/step - loss: 0.0731 - accuracy: 0.9694 - val_loss: 0.0649 - val_accuracy: 0.9775
Epoch 7/10
498/498 [==============================] - 2s 4ms/step - loss: 0.0733 - accuracy: 0.9701 - val_loss: 0.0890 - val_accuracy: 0.9730
Epoch 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [ ]:
from sklearn import metrics
y_pred = model.predict(test_features)
probas = np.array(y_pred)
labels = []
for prob in probas:
  if prob < 0.5:
    labels.append(0)
  else:
    labels.append(1)
y_true_labels = test_labels
y_pred_labels = labels
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_true_labels, y_pred_labels)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_true_labels, y_pred_labels,pos_label='positive', average='micro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_true_labels, y_pred_labels, average='micro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')
print('F1 score: %f' % f1)
from sklearn.metrics import classification_report
print(classification_report(y_true_labels, y_pred_labels, target_names=["covid", "normal"]))
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true_labels, y_pred_labels)

In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import  InceptionV3
from tensorflow.python.keras.applications.nasnet import NASNetMobile
import numpy as np

extraction_layer = NASNetMobile(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)
extraction_layer.summary()

In [ ]:
train_features, train_labels = extract_features(4800, train_generator, 20)
validation_features, validation_labels = extract_features(1200, validation_generator, 20)

In [ ]:
from keras.models import Sequential
from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(
    loss='binary_crossentropy',
    optimizer= "adam" ,
    metrics=['accuracy']
)

In [ ]:
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
learn_control = ReduceLROnPlateau(monitor='accuracy', patience=5,
                                  verbose=1,factor=0.2, min_lr=1e-7)

model_checkpoint = ModelCheckpoint('/content/gdrive/My Drive/models/nasnasnet.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [ ]:
history = model.fit(train_features, train_labels,
                    epochs=100,
                    batch_size=16,
                    validation_data=(validation_features, validation_labels),
                    callbacks=[learn_control, model_checkpoint])

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

f, axes = plt.subplots(1,2,figsize=(14,4))

axes[0].plot(epochs, acc, 'bo', label='Training acc')
axes[0].plot(epochs, val_acc, 'b', label='Validation acc')
axes[0].legend()

axes[1].plot(epochs, loss, 'bo', label='Training loss')
axes[1].plot(epochs, val_loss, 'b', label='Validation loss')
axes[1].yaxis.set_label_position("right")
axes[1].legend()

plt.show()

In [ ]:
test_features, test_labels = extract_features(1232, test_generator, 1)

In [ ]:
from sklearn import metrics
y_pred = model.predict(test_features)
probas = np.array(y_pred)
labels = []
for prob in probas:
  if prob < 0.5:
    labels.append(0)
  else:
    labels.append(1)
y_true_labels = test_labels
y_pred_labels = labels
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_true_labels, y_pred_labels)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_true_labels, y_pred_labels,pos_label='positive', average='micro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_true_labels, y_pred_labels, average='micro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')
print('F1 score: %f' % f1)
from sklearn.metrics import classification_report
print(classification_report(y_true_labels, y_pred_labels, target_names=["covid", "normal"]))
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true_labels, y_pred_labels)